# Problem-Specific Coordinate Generation for HyperNEAT Substrates

# Extra Part: Additional Random Substrates

This part was added to validate the performance of random substrates, by generating more substrates with different seeds.

## Setup

### Imports

In [1]:
import pickle
from collections import defaultdict

from config import config

from tensorneat.algorithm.hyperneat import FullSubstrate
from substrate_generation.random_coor_generator import RandomCoordinateGenerator
from substrate_generation.hidden_layers import HiddenLayerGenerator

from evol_pipeline.brax_env import CustomBraxEnv

from utils.utils import setup_folders_substrate, save_coordinates_to_csv

The folder structure is checked to avoid problems when saving files.

In [2]:
OUTPUT_DIR = config["experiment"]["output_dir_substrates"]
setup_folders_substrate(OUTPUT_DIR)

The environment needs to be setup beforehand to get obs_size and act_size.

In [3]:
env_name = config["experiment"]["env_name"]
env_problem = CustomBraxEnv(
    env_name=env_name,
    backend=config["environment"]["backend"],
    brax_args=config["environment"]["brax_args"],
    max_step=config["environment"]["max_step"],
    repeat_times=config["environment"]["repeat_times"],
    obs_normalization=config["environment"]["obs_normalization"],
    sample_episodes=config["environment"]["sample_episodes"],
)
obs_size = env_problem.input_shape[0]
act_size = env_problem.output_shape[0]
feature_dims_repeats = config["data_analysis"]["feature_dims"]

/home/andi/anaconda3/envs/jax/lib/python3.10/site-packages/brax/io/mjcf.py:480: UserWarning: Brax System, piplines and environments are not actively being maintained. Please see MJX for a well maintained JAX-based physics engine: https://github.com/google-deepmind/mujoco/tree/main/mjx. For a host of environments that use MJX, see: https://github.com/google-deepmind/mujoco_playground.
  warnings.warn(
2025-11-13 16:08:58.323097: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.
2025-11-13 16:09:09.529055: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by usin

In [4]:
feature_dims_repeats = config["data_analysis"]["feature_dims"]

def deep_defaultdict():
    return defaultdict(deep_defaultdict)

generated_io_coors = deep_defaultdict()
substrates = deep_defaultdict()

In [5]:
hidden_layer_gen = HiddenLayerGenerator(
    env_name=env_name,
    obs_size=obs_size,
    act_size=act_size,
    hidden_layer_type=config["substrate"]["hidden_layer_type"],
    hidden_depth=config["substrate"]["hidden_depth"],
    depth_factor=config["substrate"]["depth_factor"],
    width_factor=config["substrate"]["width_factor"],
)

### Random Substrate

We also want to test if our data-driven approaches are better than arbritarily chosen coordinates in an equally high-dimensional space. For this purpose the generator creates a substrate with the given number of dimensions in which all coordinates are random binary combinations, e.g. (0,1,0,1,1,0). If the target number of coordinates can't be created from unique binary combinations, the generator uses terniary combinations of 0, 0.5, 1. If that also fails to generate the required number of coordinates, if falls back to continous random numbers.

In [6]:
seeds = [231,10319]

for seed in seeds:
    for feature_dim in feature_dims_repeats:  
        random_generator = RandomCoordinateGenerator(
            obs_size=obs_size,
            act_size=act_size,
            feature_dims=feature_dim,
            depth_factor=config["substrate"]["depth_factor"],
            width_factor=config["substrate"]["width_factor"],
            seed=seed,
        )
        input_coors, output_coors = random_generator.generate_io_coordinates()
        hidden_coors = hidden_layer_gen.get_hidden_coors(input_coors=input_coors)
        substrates["no_data"]["random_coors"][feature_dim][f"{seed}"]["input_coors"] = input_coors
        substrates["no_data"]["random_coors"][feature_dim][f"{seed}"]["hidden_coors"] = hidden_coors
        substrates["no_data"]["random_coors"][feature_dim][f"{seed}"]["output_coors"] = output_coors

        substrates["no_data"]["random_coors"][feature_dim][f"{seed}"]["substrate"] = FullSubstrate(
            input_coors=input_coors,
            hidden_coors=hidden_coors,
            output_coors=output_coors
        )

        log_coors = [*input_coors, *hidden_coors, *output_coors]
        save_coordinates_to_csv(
            coordinates=log_coors,
            filepath=f"{OUTPUT_DIR}/coordinates/no_data_random_coors_{feature_dim}_seed{seed}_io.csv",
        )

Generating random coordinates with 24 feature dimensions...
Added layering dimension. Final coordinate size: 25
Successfully saved coordinates to: output_substrates/coordinates/no_data_random_coors_24_seed231_io.csv
Generating random coordinates with 24 feature dimensions...
Added layering dimension. Final coordinate size: 25
Successfully saved coordinates to: output_substrates/coordinates/no_data_random_coors_24_seed10319_io.csv


### Substrate from Data Analysis Results

Next, the input and output nodes created through data analysis are used to create substrates.

The substrates are saved to disk to use in other notebooks.

In [7]:
with open("random_substrates_d25.pkl", "wb") as f:
    pickle.dump(substrates, f)

In [8]:
import os

import wandb

from config import config

import jax
from evol_pipeline.custom_pipeline import CustomPipeline
from evol_pipeline.evol_algorithm import create_evol_algorithm

from utils.visualization import visualize_cppn, visualize_nn
from utils.utils import setup_folders_evolution, append_summary_row

In [9]:
OUTPUT_DIR = config["experiment"]["output_dir_evolution"]
setup_folders_evolution(OUTPUT_DIR)

In [10]:
for feature_dims_label, feature_dims_dict in substrates["no_data"]["random_coors"].items():
    for seed, seed_dict in feature_dims_dict.items():


        active_substrate = seed_dict["substrate"]
        evol_algorithm = create_evol_algorithm(substrate=active_substrate)
        substrate_dimensions = int(active_substrate.query_coors.shape[1]/2)
        initial_cppn_layers = config["algorithm"]["genome"]["cppn_init_hidden_layers"](active_substrate.query_coors.shape[1])

        run_id = f"no_data_random_coors_{substrate_dimensions}d_seed{seed}"

        wanbd_name = run_id
        wandb_tags = [config["substrate"]["hidden_layer_type"], env_name, "no_data", "random_coors", f"{config['substrate']['hidden_depth']}_hl", f"{config['algorithm']['neat']['pop_size']}pop", f"{config['environment']['backend']}"]

        wandb.init(
            name=wanbd_name,
            project="substrates",
            tags=wandb_tags,
            config=config  
        )

        wandb.config.update(
            {
                "substrate": {
                    "obs_size": obs_size,
                    "act_size": act_size,
                    "num_queries": active_substrate.query_coors.shape[0],
                    "query_dim": active_substrate.query_coors.shape[1],
                    },
                "algorithm": {
                    "neat": {
                        "num_inputs": evol_algorithm.num_inputs,
                        },
                    "genome": {
                        "cppn_init_hidden_layers": initial_cppn_layers,
                        },
                    },
                "random_coor_seed": seed,
            },
        )
        
        genome_save_dir = os.path.join(OUTPUT_DIR, "genome", run_id)
        os.makedirs(genome_save_dir, exist_ok=True)

        pipeline = CustomPipeline(
            algorithm=evol_algorithm,
            problem=env_problem,
            seed=config["experiment"]["seed"],
            generation_limit=config["pipeline"]["generation_limit"],
            fitness_target=config["pipeline"]["fitness_target"],
            is_save=True,
            save_dir=genome_save_dir,
        )

        init_state = pipeline.setup()
        train_state, best_genome = pipeline.auto_run(state=init_state)

        print(f"\nTraining finished. Best fitness achieved: {pipeline.best_fitness}")

        wandb.finish()

        append_summary_row(
            path=f"{OUTPUT_DIR}/summary_results.csv",
            row={
                "dimensionality": int(active_substrate.query_coors.shape[1] / 2),
                "sampling": "no_data",
                "method": "random_coors",
                "max_fitness": float(pipeline.best_fitness),
            },
        )

        # Uses the trained state for parameters, but overrides the key for reset/render. This way all videos start in the same state for comparison.
        k_show = jax.random.PRNGKey(config["experiment"]["seed"]+5)
        show_state = train_state.update(randkey=k_show)

        # Built-in show method to produce and save a video of the agent
        pipeline.show(
            state=show_state,
            best=best_genome,
            output_type="mp4",
            save_path=f"{OUTPUT_DIR}/video/agent_no_data_random_coors_{feature_dims_label}_seed{seed}.mp4",
        )

        # All coordinates are needed for visualization
        input_coors = seed_dict["input_coors"]
        hidden_coors = seed_dict["hidden_coors"]
        output_coors = seed_dict["output_coors"]

        # Visualizes the CPPN
        visualize_cppn(
            pipeline=pipeline, 
            state=train_state, 
            save_path=f"{OUTPUT_DIR}/topology/cppn_no_data_random_coors_{feature_dims_label}_seed{seed}.svg"
        )
        
        # Visualizes a representation of the neural network in 2D space
        visualize_nn(
            pipeline=pipeline, 
            state=train_state, 
            save_path=f"{OUTPUT_DIR}/topology/nn_no_data_random_coors_{feature_dims_label}_seed{seed}.svg", 
            substrate=active_substrate, 
            input_coors=input_coors, 
            hidden_coors=hidden_coors, 
            output_coors=output_coors, 
            hidden_depth=config["substrate"]["hidden_depth"], 
            max_weight=config["algorithm"]["hyperneat"]["max_weight"], 
        )

wandb: Currently logged in as: wirkelzirkel (wirkelzirkel-iu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


save to output_evolution/genome/no_data_random_coors_25d_seed231
initializing
initializing finished
start compile
compile finished, cost time: 29.893128s
Generation: 1, Cost time: 5039.65ms
 	fitness: valid cnt: 500, max: 50.8320, min: -3954.1860, mean: -1228.8158, std: 1277.1610

	node counts: max: 54, min: 52, mean: 53.27
 	conn counts: max: 104, min: 51, mean: 101.97
 	species: 10, [4, 79, 82, 11, 24, 12, 16, 30, 15, 227]

Generation: 2, Cost time: 4959.20ms
 	fitness: valid cnt: 500, max: 52.1594, min: -3926.9456, mean: -305.0605, std: 771.6681

	node counts: max: 55, min: 52, mean: 53.43
 	conn counts: max: 105, min: 50, mean: 97.88
 	species: 10, [7, 79, 42, 71, 11, 4, 56, 25, 11, 194]

Generation: 3, Cost time: 5041.91ms
 	fitness: valid cnt: 500, max: 53.0762, min: -3500.2769, mean: -77.1256, std: 463.9534

	node counts: max: 56, min: 52, mean: 53.82
 	conn counts: max: 107, min: 51, mean: 102.73
 	species: 10, [10, 1, 63, 49, 58, 43, 59, 19, 61, 137]

Generation: 4, Cost time:

compute_ms,▆▇▃▃▁▇▇▇▅██▁▃▂▂▂▃▅▅▃▄▅▄▆▅▃▃▆▃▆▆▅▇▆▆▅▆▆▆▅
compute_ms_pop,█▃▄█▅▄▁▄▁▂▅▅▂▅▃▁▆▃▅▂▄▄▃▅▃▃▆▅▆▇█▆▆▆▆▇▇▇▇▇
fitness_max,▁▃▃▃▄▅▄▃▄▅▄▄▅▅▅▆▆▆▇▅▆▆▆▆▅▆▇▆▆▇▇▇▆▆▆▆▇▇▇█
fitness_mean,▆█▆▅▅▅▄▅▆▅▆▅▇▅▇▂▆▆▄█▂▄▆▂▇▄▄▅▄▅▆▆▆▅▇▃▅▁▆▄
fitness_min,▅▁█▁▁▂▁▂█▁█▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁█▁▁▁▁█▁██▁▁█
fitness_std,▃▅▅▄▃▃▁▄▄▅▃▆▆▆▇▆▅▆▆▅▆▇▆▅▅▇▇▅▆▆▆▇▇▆▆▆▆█▅▇
generation,▁▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇███
num_species,███▇▇████▆▆▅▄▄▄▅▆▅▆▆▅▃▃▃▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁
compute_ms,5022.17746
compute_ms_pop,10.04435
fitness_max,485.19458


2025-11-13 16:50:30.183839: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.
2025-11-13 16:50:30.183855: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.


Total reward:  569.17584


...

save to output_evolution/genome/no_data_random_coors_25d_seed10319
initializing
initializing finished
start compile
compile finished, cost time: 29.957201s
Generation: 1, Cost time: 4728.22ms
 	fitness: valid cnt: 500, max: 54.9948, min: -3953.0532, mean: -1240.2507, std: 1282.8383

	node counts: max: 54, min: 52, mean: 53.27
 	conn counts: max: 104, min: 51, mean: 101.97
 	species: 10, [4, 78, 82, 12, 24, 12, 16, 33, 20, 219]

Generation: 2, Cost time: 4677.18ms
 	fitness: valid cnt: 500, max: 54.3575, min: -3953.6047, mean: -243.9969, std: 688.0464

	node counts: max: 55, min: 52, mean: 53.61
 	conn counts: max: 106, min: 3, mean: 99.24
 	species: 10, [83, 81, 1, 39, 57, 39, 6, 23, 14, 157]

Generation: 3, Cost time: 4708.64ms
 	fitness: valid cnt: 500, max: 56.1658, min: -3929.6182, mean: -134.6917, std: 638.4860

	node counts: max: 56, min: 52, mean: 53.79
 	conn counts: max: 106, min: 50, mean: 101.69
 	species: 10, [86, 1, 76, 59, 1, 27, 37, 17, 20, 176]

Generation: 4, Cost time

compute_ms,▁▁▂▂▂▃▁▃▃▃▃▃▃▂▂▃▄▄▄▄▅▄▅▆▆▆▅▅▅▆▆▇█▅▇▆▆▆▆▅
compute_ms_pop,▁▂▁▁▁▂▃▃▃▃▂▂▂▄▄▅▅▄▄▆▇▇▇▇▇▆▇▇██▆█▇▇▇▇▆▆▇█
fitness_max,▁▂▂▂▂▄▄▄▄▆▆▆▅▅▅▆▆▆▇▇▇▇▇▇▇█▇▆▇▇▇▇▇▇▇▇▇▇▇█
fitness_mean,▁▃▃▄▄▅▆▆▅▅▅▆▆█▇▆▇▇▆▆▇▇██▆▆▆▇▇▆█▇█▇▇▆▆▆▇▇
fitness_min,▃▁▂▃▁▃▆▁▁▄▄▁▃▃▁▄▃▄▃▃█▄▃▁▄▃▄▅▃▁▁▄▅▁▃▆▁▃▂▇
fitness_std,█▃▆▄▂▅▂▃▄▁▂▂▃▂▁▃▃▂▃▂▂▂▁▁▃▃▃▃▃▄▃▃▃▃▂▃▅▄▄▄
generation,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇███
num_species,█████▇▆▅▄▃▂▃▃▂▃▁▂▃▂▃▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
compute_ms,4992.94281
compute_ms_pop,9.98589
fitness_max,482.71436


Total reward:  435.2743


/home/andi/anaconda3/envs/jax/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


mp4 saved to:  output_evolution/video/agent_no_data_random_coors_24_seed10319.mp4
Visualizing CPPN. Saving to output_evolution/topology/cppn_no_data_random_coors_24_seed10319.svg.
Manually reconstructing the phenotype. A visual layout will be generated.
Substrate has 1792 potential connections.
Visualizing 1748 connections. Excluded loops. Weight threshold: 0.005
Visualization saved to: output_evolution/topology/nn_no_data_random_coors_24_seed10319.svg
